# Analytics of MobileNet V1 Model Sizes


In [1]:
import os, sys, math, datetime
import psutil

# import pathlib
from pathlib import Path
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot as plt
import PIL
import PIL.Image

# Import the necessary MLTK APIs
# from mltk.core import view_model, summarize_model, profile_model
# from mltk.utils.commander import query_platform

# import tensorflow as tf
# import tensorflow_datasets as tfds
# from tensorflow import keras
# from tensorflow.keras.layers import (
#     Input,
#     Dense,
#     Flatten,
#     Conv2D,
#     MaxPooling2D,
#     GlobalAveragePooling2D,
#     BatchNormalization,
#     Concatenate,
# )
# from tensorflow.keras.models import Model
# from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# import workbench.config.config
from workbench.config.config import initialize
from workbench.utils.utils import create_filepaths

import plotly.express as px
# enable plotly in VS Studio Code
import plotly.io as pio
pio.renderers.default = "notebook_connected"

import wandb


In [2]:
# Configure pandas to show all columns & rows
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [3]:
models_dir = initialize()

# Load csv file

In [4]:
base_model_name = "mobilenetv1"

In [5]:
csv_file_name = f"{base_model_name}_stats.csv"
csv_path = Path.cwd().joinpath(csv_file_name)

In [110]:
df = pd.read_csv(csv_path)

# EDA

In [124]:
df["total_params_kB"] = df["total_params"] * 4 / 1024
df["tflite_INT8_model_size_kB"] = df["tflite_model_size_kb"] / 4
#df.insert(5,"tflite_INT8_model_size_kb" , df.pop("tflite_INT8_model_size_kb"))
#df.insert(14,"dropout_rate" , df.pop("dropout_rate"))
df.sort_values(["tflite_INT8_model_size_kb", "model_name"], inplace=True)
df.head(10)


,model_name,params_kb,MACs_kB,FLOPs_kB,img_res,tflite_INT8_model_size_kb,classes,channels,alpha,MACs,FLOPs,total_params,trainable_params,non_trainable_params,dropout_rate,model_size_kb,tflite_model_size_kb,total_params_kb,total_params_kB,tflite_INT8_model_size_kB
14,mobilenetv1_0.1_96_c3_o3_l1,105.445312,1030.144,2288.640,96,26.911133,3,3,0.1,1.006 M,2.235 M,26994,25642,1352,0.2,310.777344,107.644531,105.445312,105.445312,26.911133
19,mobilenetv1_0.1_96_c3_o3_l2,119.390625,1143.808,2525.184,96,30.218750,3,3,0.1,1.117 M,2.466 M,30564,29008,1556,0.2,323.992188,120.875000,119.390625,119.390625,30.218750
13,mobilenetv1_0.1_96_c3_o3_keras,156.992188,1481.728,3145.728,96,39.896484,3,3,0.1,1.447 M,3.072 M,40190,38022,2168,0.2,410.996094,159.585938,156.992188,156.992188,39.896484
9,mobilenetv1_0.1_128_c3_o3_keras,156.992188,2633.728,5591.040,128,39.947266,3,3,0.1,2.572 M,5.460 M,40190,38022,2168,0.2,411.003906,159.789062,156.992188,156.992188,39.947266
5,mobilenetv1_0.1_224_c3_o3_keras,156.992188,8066.048,17122.304,224,39.947266,3,3,0.1,7.877 M,16.721 M,40190,38022,2168,0.2,411.003906,159.789062,156.992188,156.992188,39.947266
15,mobilenetv1_0.2_96_c3_o3_l1,373.757812,3320.832,7104.512,96,90.873047,3,3,0.2,3.243 M,6.938 M,95682,92962,2720,0.2,580.339844,363.492188,373.757812,373.757812,90.873047
20,mobilenetv1_0.2_96_c3_o3_l2,421.968750,3738.624,7961.600,96,102.249023,3,3,0.2,3.651 M,7.775 M,108024,104896,3128,0.2,627.277344,408.996094,421.968750,421.968750,102.249023
12,mobilenetv1_0.2_96_c3_o3_keras,558.101562,4990.976,10349.568,96,136.135742,3,3,0.2,4.874 M,10.107 M,142874,138522,4352,0.2,812.339844,544.542969,558.101562,558.101562,136.135742
8,mobilenetv1_0.2_128_c3_o3_keras,558.101562,8871.936,18398.208,128,136.187500,3,3,0.2,8.664 M,17.967 M,142874,138522,4352,0.2,812.347656,544.750000,558.101562,558.101562,136.187500
4,mobilenetv1_0.2_224_c3_o3_keras,558.101562,27168.768,56341.504,224,136.187500,3,3,0.2,26.532 M,55.021 M,142874,138522,4352,0.2,812.347656,544.750000,558.101562,558.101562,136.187500


In [125]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 14 to 0
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   model_name                 24 non-null     object 
 1   params_kb                  24 non-null     float64
 2   MACs_kB                    24 non-null     float64
 3   FLOPs_kB                   24 non-null     float64
 4   img_res                    24 non-null     int64  
 5   tflite_INT8_model_size_kb  24 non-null     float64
 6   classes                    24 non-null     int64  
 7   channels                   24 non-null     int64  
 8   alpha                      24 non-null     float64
 9   MACs                       24 non-null     object 
 10  FLOPs                      24 non-null     object 
 11  total_params               24 non-null     int64  
 12  trainable_params           24 non-null     int64  
 13  non_trainable_params       24 non-null     int64  
 

# Helper functions

In [127]:
def convert_to_kb(x):
    input_value = x["input_value"]
    unit = x["unit"]
    if unit=="k":
        kb_value = input_value
    elif unit=="M":
        kb_value = input_value * 1024
    elif  unit=="G":
        kb_value = input_value * 1024 * 1024
    else:
        kb_value ="NaN"
    return kb_value

In [128]:
def create_kb_column(df, column_name):
    df_cols = df[column_name].str.split(" ", expand=True)
    df_cols.columns =["input_value", "unit"]
    df_cols["input_value"] = df_cols["input_value"].astype(float)
    df_cols["kB"] = df_cols.apply(convert_to_kb, axis=1)
    return df_cols["kB"]

# Feature generation & data cleaning

In [129]:
df["MACs_kB"] = create_kb_column(df, "MACs")
df["FLOPs_kB"] = create_kb_column(df, "FLOPs")
df.head()

,model_name,params_kb,MACs_kB,FLOPs_kB,img_res,tflite_INT8_model_size_kb,classes,channels,alpha,MACs,FLOPs,total_params,trainable_params,non_trainable_params,dropout_rate,model_size_kb,tflite_model_size_kb,total_params_kb,total_params_kB,tflite_INT8_model_size_kB
14,mobilenetv1_0.1_96_c3_o3_l1,105.445312,1030.144,2288.640,96,26.911133,3,3,0.1,1.006 M,2.235 M,26994,25642,1352,0.2,310.777344,107.644531,105.445312,105.445312,26.911133
19,mobilenetv1_0.1_96_c3_o3_l2,119.390625,1143.808,2525.184,96,30.218750,3,3,0.1,1.117 M,2.466 M,30564,29008,1556,0.2,323.992188,120.875000,119.390625,119.390625,30.218750
13,mobilenetv1_0.1_96_c3_o3_keras,156.992188,1481.728,3145.728,96,39.896484,3,3,0.1,1.447 M,3.072 M,40190,38022,2168,0.2,410.996094,159.585938,156.992188,156.992188,39.896484
9,mobilenetv1_0.1_128_c3_o3_keras,156.992188,2633.728,5591.040,128,39.947266,3,3,0.1,2.572 M,5.460 M,40190,38022,2168,0.2,411.003906,159.789062,156.992188,156.992188,39.947266
5,mobilenetv1_0.1_224_c3_o3_keras,156.992188,8066.048,17122.304,224,39.947266,3,3,0.1,7.877 M,16.721 M,40190,38022,2168,0.2,411.003906,159.789062,156.992188,156.992188,39.947266


In [144]:
df.insert(0, 'model_name', df.pop('model_name'))
df.insert(1, 'tflite_INT8_model_size_kb', df.pop('tflite_INT8_model_size_kb'))
df.insert(2, 'total_params_kB', df.pop('total_params_kB'))
df.insert(3, 'MACs_kB', df.pop('MACs_kB'))
df.insert(4, 'FLOPs_kB', df.pop('FLOPs_kB'))
df.sort_values("model_name", inplace=True)
df.head()

,model_name,tflite_INT8_model_size_kb,total_params_kB,MACs_kB,FLOPs_kB,params_kb,img_res,classes,channels,alpha,MACs,FLOPs,total_params,trainable_params,non_trainable_params,dropout_rate,model_size_kb,tflite_model_size_kb,total_params_kb,tflite_INT8_model_size_kB
9,mobilenetv1_0.1_128_c3_o3_keras,39.947266,156.992188,2633.728,5591.040,156.992188,128,3,3,0.1,2.572 M,5.460 M,40190,38022,2168,0.2,411.003906,159.789062,156.992188,39.947266
5,mobilenetv1_0.1_224_c3_o3_keras,39.947266,156.992188,8066.048,17122.304,156.992188,224,3,3,0.1,7.877 M,16.721 M,40190,38022,2168,0.2,411.003906,159.789062,156.992188,39.947266
13,mobilenetv1_0.1_96_c3_o3_keras,39.896484,156.992188,1481.728,3145.728,156.992188,96,3,3,0.1,1.447 M,3.072 M,40190,38022,2168,0.2,410.996094,159.585938,156.992188,39.896484
14,mobilenetv1_0.1_96_c3_o3_l1,26.911133,105.445312,1030.144,2288.640,105.445312,96,3,3,0.1,1.006 M,2.235 M,26994,25642,1352,0.2,310.777344,107.644531,105.445312,26.911133
19,mobilenetv1_0.1_96_c3_o3_l2,30.218750,119.390625,1143.808,2525.184,119.390625,96,3,3,0.1,1.117 M,2.466 M,30564,29008,1556,0.2,323.992188,120.875000,119.390625,30.218750


In [145]:
def highlight_cells(val):
    threshold = 1000
    color = 'green' if val < threshold else 'red'
    return f'background-color: {color}'

In [146]:
df.style.applymap(highlight_cells,
    subset=["tflite_INT8_model_size_kb"])
#df

,model_name,tflite_INT8_model_size_kb,total_params_kB,MACs_kB,FLOPs_kB,params_kb,img_res,classes,channels,alpha,MACs,FLOPs,total_params,trainable_params,non_trainable_params,dropout_rate,model_size_kb,tflite_model_size_kb,total_params_kb,tflite_INT8_model_size_kB
9,mobilenetv1_0.1_128_c3_o3_keras,39.947266,156.992188,2633.728000,5591.040000,156.992188,128,3,3,0.100000,2.572 M,5.460 M,40190,38022,2168,0.200000,411.003906,159.789062,156.992188,39.947266
5,mobilenetv1_0.1_224_c3_o3_keras,39.947266,156.992188,8066.048000,17122.304000,156.992188,224,3,3,0.100000,7.877 M,16.721 M,40190,38022,2168,0.200000,411.003906,159.789062,156.992188,39.947266
13,mobilenetv1_0.1_96_c3_o3_keras,39.896484,156.992188,1481.728000,3145.728000,156.992188,96,3,3,0.100000,1.447 M,3.072 M,40190,38022,2168,0.200000,410.996094,159.585938,156.992188,39.896484
14,mobilenetv1_0.1_96_c3_o3_l1,26.911133,105.445312,1030.144000,2288.640000,105.445312,96,3,3,0.100000,1.006 M,2.235 M,26994,25642,1352,0.200000,310.777344,107.644531,105.445312,26.911133
19,mobilenetv1_0.1_96_c3_o3_l2,30.218750,119.390625,1143.808000,2525.184000,119.390625,96,3,3,0.100000,1.117 M,2.466 M,30564,29008,1556,0.200000,323.992188,120.875000,119.390625,30.218750
7,mobilenetv1_0.25_128_c3_o3_keras,208.750977,856.699219,13634.560000,28111.872000,856.699219,128,3,3,0.250000,13.315 M,27.453 M,219315,213843,5472,0.200000,1109.796875,835.003906,856.699219,208.750977
3,mobilenetv1_0.25_224_c3_o3_keras,208.750977,856.699219,41753.600000,86088.704000,856.699219,224,3,3,0.250000,40.775 M,84.071 M,219315,213843,5472,0.200000,1109.796875,835.003906,856.699219,208.750977
11,mobilenetv1_0.25_96_c3_o3_keras,208.699219,856.699219,7669.760000,15813.632000,856.699219,96,3,3,0.250000,7.490 M,15.443 M,219315,213843,5472,0.200000,1109.789062,834.796875,856.699219,208.699219
16,mobilenetv1_0.25_96_c3_o3_l1,139.163086,573.386719,5084.160000,10766.336000,573.386719,96,3,3,0.250000,4.965 M,10.514 M,146787,143363,3424,0.200000,779.328125,556.652344,573.386719,139.163086
21,mobilenetv1_0.25_96_c3_o3_l2,156.607422,646.886719,5730.304000,12087.296000,646.886719,96,3,3,0.250000,5.596 M,11.804 M,165603,161667,3936,0.200000,851.906250,626.429688,646.886719,156.607422


In [9]:
len(df)

24

In [105]:
params_df = df.copy()

In [106]:
params_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 14 to 0
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   img_res                    24 non-null     int64  
 1   classes                    24 non-null     int64  
 2   channels                   24 non-null     int64  
 3   alpha                      24 non-null     float64
 4   model_name                 24 non-null     object 
 5   tflite_INT8_model_size_kb  24 non-null     float64
 6   MACs                       24 non-null     object 
 7   FLOPs                      24 non-null     object 
 8   total_params               24 non-null     int64  
 9   trainable_params           24 non-null     int64  
 10  non_trainable_params       24 non-null     int64  
 11  model_size_kb              24 non-null     float64
 12  tflite_model_size_kb       24 non-null     float64
 13  params_kb                  24 non-null     float64
 

In [10]:
df_c1 = df.loc[(df['channels'] == 1)]
df_c1_sorted = df_c1.sort_values(['img_res', 'tflite_INT8_model_size_kb'],
              ascending = [True, True])
df_c1_sorted

,img_res,classes,channels,alpha,tflite_INT8_model_size_kb,model_name,MACs,FLOPs,total_params,trainable_params,non_trainable_params,model_size_kb,tflite_model_size_kb,params_kb,dropout_rate


In [11]:
df_c3 = df.loc[(df['channels'] == 3)]
df_c3_sorted = df_c3.sort_values(['img_res', 'tflite_INT8_model_size_kb'],
              ascending = [True, True])
df_c3_sorted

,img_res,classes,channels,alpha,tflite_INT8_model_size_kb,model_name,MACs,FLOPs,total_params,trainable_params,non_trainable_params,model_size_kb,tflite_model_size_kb,params_kb,dropout_rate
14,96,3,3,0.10,26.911133,mobilenetv1_0.1_96_c3_o3_l1,1.006 M,2.235 M,26994,25642,1352,310.777344,107.644531,105.445312,0.2
19,96,3,3,0.10,30.218750,mobilenetv1_0.1_96_c3_o3_l2,1.117 M,2.466 M,30564,29008,1556,323.992188,120.875000,119.390625,0.2
13,96,3,3,0.10,39.896484,mobilenetv1_0.1_96_c3_o3_keras,1.447 M,3.072 M,40190,38022,2168,410.996094,159.585938,156.992188,0.2
15,96,3,3,0.20,90.873047,mobilenetv1_0.2_96_c3_o3_l1,3.243 M,6.938 M,95682,92962,2720,580.339844,363.492188,373.757812,0.2
20,96,3,3,0.20,102.249023,mobilenetv1_0.2_96_c3_o3_l2,3.651 M,7.775 M,108024,104896,3128,627.277344,408.996094,421.968750,0.2
12,96,3,3,0.20,136.135742,mobilenetv1_0.2_96_c3_o3_keras,4.874 M,10.107 M,142874,138522,4352,812.339844,544.542969,558.101562,0.2
16,96,3,3,0.25,139.163086,mobilenetv1_0.25_96_c3_o3_l1,4.965 M,10.514 M,146787,143363,3424,779.328125,556.652344,573.386719,0.2
21,96,3,3,0.25,156.607422,mobilenetv1_0.25_96_c3_o3_l2,5.596 M,11.804 M,165603,161667,3936,851.906250,626.429688,646.886719,0.2
17,96,3,3,0.30,195.917969,mobilenetv1_0.3_96_c3_o3_l1,6.724 M,14.135 M,206471,202381,4090,1014.105469,783.671875,806.527344,0.2
11,96,3,3,0.25,208.699219,mobilenetv1_0.25_96_c3_o3_keras,7.490 M,15.443 M,219315,213843,5472,1109.789062,834.796875,856.699219,0.2


# Get experiment data from W&B

In [33]:
api = wandb.Api()

In [38]:
# Project is specified by <entity/project-name>
runs = api.runs("susbrock/mobilenetv1")

summary_list, config_list, name_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    name_list.append((run.name, run.id))

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "name": name_list
    })

#runs_df.to_csv("project.csv")

In [40]:
name_df = pd.DataFrame(name_list, columns=["name", "id"])
name_df

,name,id
0,smooth-dew-21,3ac53h9j
1,dandy-universe-20,2w85dvzm
2,deft-fog-19,2wgi6x5i
3,wobbly-brook-18,1jqfxmbf
4,effortless-frost-17,1tkw3yhl
5,mobilenetv1_0.5_96_c3_o3_l2,2csc8z41
6,mobilenetv1_0.3_96_c3_o3_l2,2tm8ysb4
7,mobilenetv1_0.25_96_c3_o3_l2,2a4a0fww
8,mobilenetv1_0.2_96_c3_o3_l2,277skkd9
9,mobilenetv1_0.1_96_c3_o3_l2,bc6hpasq


In [34]:
config_df = pd.DataFrame(config_list)
config_df

,id,epochs,classes,momentum,optimizer,batch_size,learn_rate,architecture
0,3ac53h9j,100.0,3.0,0.9,SGD,32.0,0.001,mobilenetv1_0.5_96_c3_o3_l2
1,2w85dvzm,100.0,3.0,0.9,SGD,32.0,0.001,mobilenetv1_0.3_96_c3_o3_l2
2,2wgi6x5i,100.0,3.0,0.9,SGD,32.0,0.001,mobilenetv1_0.25_96_c3_o3_l2
3,1jqfxmbf,100.0,3.0,0.9,SGD,32.0,0.001,mobilenetv1_0.2_96_c3_o3_l2
4,1tkw3yhl,100.0,3.0,0.9,SGD,32.0,0.001,mobilenetv1_0.1_96_c3_o3_l2
5,2csc8z41,NaN,NaN,NaN,NaN,NaN,NaN,mobilenetv1_0.5_96_c3_o3_l2
6,2tm8ysb4,NaN,NaN,NaN,NaN,NaN,NaN,mobilenetv1_0.3_96_c3_o3_l2
7,2a4a0fww,NaN,NaN,NaN,NaN,NaN,NaN,mobilenetv1_0.25_96_c3_o3_l2
8,277skkd9,NaN,NaN,NaN,NaN,NaN,NaN,mobilenetv1_0.2_96_c3_o3_l2
9,bc6hpasq,NaN,NaN,NaN,NaN,NaN,NaN,mobilenetv1_0.1_96_c3_o3_l2


In [35]:
summary_df = pd.DataFrame(summary_list)
summary_df

,epoch/loss,epoch/val_accuracy,epoch/epoch,conf_mat_table,epoch/accuracy,epoch/val_loss,_step,_wandb,_runtime,_timestamp,epoch/learning_rate,Chart_table
0,0.011499,0.948413,86.0,{'sha256': '194f77e76a1b7785b6b031759c8e0489c6...,0.996536,0.135964,87,{'runtime': 385},386.162590,1.671313e+09,0.001,NaN
1,0.012104,0.984127,76.0,{'_latest_artifact_path': 'wandb-client-artifa...,0.996536,0.040888,77,{'runtime': 296},297.806900,1.671312e+09,0.001,NaN
2,0.011761,0.976190,91.0,"{'_type': 'table-file', 'ncols': 3, 'nrows': 9...",0.996042,0.067346,92,{'runtime': 379},380.362510,1.671312e+09,0.001,NaN
3,0.011326,0.980159,99.0,"{'size': 435, '_type': 'table-file', 'ncols': ...",0.997526,0.078373,100,{'runtime': 415},416.263778,1.671311e+09,0.001,NaN
4,0.024092,0.972222,54.0,{'sha256': '216f09a4c259c1edb7bb9a5657ad753975...,0.992578,0.065462,55,{'runtime': 220},220.904655,1.671311e+09,0.001,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,0,{'runtime': 10},7.737342,1.671311e+09,NaN,{'sha256': '1fa1fcdfc7cf2b004b0af91e3403cecfd1...
6,NaN,NaN,NaN,NaN,NaN,NaN,0,{'runtime': 10},7.677541,1.671311e+09,NaN,{'_latest_artifact_path': 'wandb-client-artifa...
7,NaN,NaN,NaN,NaN,NaN,NaN,0,{'runtime': 10},7.739293,1.671311e+09,NaN,"{'_type': 'table-file', 'ncols': 1, 'nrows': 1..."
8,NaN,NaN,NaN,NaN,NaN,NaN,0,{'runtime': 10},7.674031,1.671311e+09,NaN,{'sha256': '1fa1fcdfc7cf2b004b0af91e3403cecfd1...
9,NaN,NaN,NaN,NaN,NaN,NaN,0,{'runtime': 10},7.924718,1.671311e+09,NaN,{'_latest_artifact_path': 'wandb-client-artifa...


In [60]:
model_df = pd.concat([name_df,  config_df, summary_df], axis=1)
#model_df = name_df.concatenate(config_df)#
#model_df = model_df.join(summary_df)
model_df

,name,id,id,epochs,classes,momentum,optimizer,batch_size,learn_rate,architecture,epoch/loss,epoch/val_accuracy,epoch/epoch,conf_mat_table,epoch/accuracy,epoch/val_loss,_step,_wandb,_runtime,_timestamp,epoch/learning_rate,Chart_table
0,smooth-dew-21,3ac53h9j,3ac53h9j,100.0,3.0,0.9,SGD,32.0,0.001,mobilenetv1_0.5_96_c3_o3_l2,0.011499,0.948413,86.0,{'sha256': '194f77e76a1b7785b6b031759c8e0489c6...,0.996536,0.135964,87,{'runtime': 385},386.162590,1.671313e+09,0.001,NaN
1,dandy-universe-20,2w85dvzm,2w85dvzm,100.0,3.0,0.9,SGD,32.0,0.001,mobilenetv1_0.3_96_c3_o3_l2,0.012104,0.984127,76.0,{'_latest_artifact_path': 'wandb-client-artifa...,0.996536,0.040888,77,{'runtime': 296},297.806900,1.671312e+09,0.001,NaN
2,deft-fog-19,2wgi6x5i,2wgi6x5i,100.0,3.0,0.9,SGD,32.0,0.001,mobilenetv1_0.25_96_c3_o3_l2,0.011761,0.976190,91.0,"{'_type': 'table-file', 'ncols': 3, 'nrows': 9...",0.996042,0.067346,92,{'runtime': 379},380.362510,1.671312e+09,0.001,NaN
3,wobbly-brook-18,1jqfxmbf,1jqfxmbf,100.0,3.0,0.9,SGD,32.0,0.001,mobilenetv1_0.2_96_c3_o3_l2,0.011326,0.980159,99.0,"{'size': 435, '_type': 'table-file', 'ncols': ...",0.997526,0.078373,100,{'runtime': 415},416.263778,1.671311e+09,0.001,NaN
4,effortless-frost-17,1tkw3yhl,1tkw3yhl,100.0,3.0,0.9,SGD,32.0,0.001,mobilenetv1_0.1_96_c3_o3_l2,0.024092,0.972222,54.0,{'sha256': '216f09a4c259c1edb7bb9a5657ad753975...,0.992578,0.065462,55,{'runtime': 220},220.904655,1.671311e+09,0.001,NaN
5,mobilenetv1_0.5_96_c3_o3_l2,2csc8z41,2csc8z41,NaN,NaN,NaN,NaN,NaN,NaN,mobilenetv1_0.5_96_c3_o3_l2,NaN,NaN,NaN,NaN,NaN,NaN,0,{'runtime': 10},7.737342,1.671311e+09,NaN,{'sha256': '1fa1fcdfc7cf2b004b0af91e3403cecfd1...
6,mobilenetv1_0.3_96_c3_o3_l2,2tm8ysb4,2tm8ysb4,NaN,NaN,NaN,NaN,NaN,NaN,mobilenetv1_0.3_96_c3_o3_l2,NaN,NaN,NaN,NaN,NaN,NaN,0,{'runtime': 10},7.677541,1.671311e+09,NaN,{'_latest_artifact_path': 'wandb-client-artifa...
7,mobilenetv1_0.25_96_c3_o3_l2,2a4a0fww,2a4a0fww,NaN,NaN,NaN,NaN,NaN,NaN,mobilenetv1_0.25_96_c3_o3_l2,NaN,NaN,NaN,NaN,NaN,NaN,0,{'runtime': 10},7.739293,1.671311e+09,NaN,"{'_type': 'table-file', 'ncols': 1, 'nrows': 1..."
8,mobilenetv1_0.2_96_c3_o3_l2,277skkd9,277skkd9,NaN,NaN,NaN,NaN,NaN,NaN,mobilenetv1_0.2_96_c3_o3_l2,NaN,NaN,NaN,NaN,NaN,NaN,0,{'runtime': 10},7.674031,1.671311e+09,NaN,{'sha256': '1fa1fcdfc7cf2b004b0af91e3403cecfd1...
9,mobilenetv1_0.1_96_c3_o3_l2,bc6hpasq,bc6hpasq,NaN,NaN,NaN,NaN,NaN,NaN,mobilenetv1_0.1_96_c3_o3_l2,NaN,NaN,NaN,NaN,NaN,NaN,0,{'runtime': 10},7.924718,1.671311e+09,NaN,{'_latest_artifact_path': 'wandb-client-artifa...


In [65]:
architectures = model_df.sort_values("architecture")
architectures.head()

,name,id,id,epochs,classes,momentum,optimizer,batch_size,learn_rate,architecture,epoch/loss,epoch/val_accuracy,epoch/epoch,conf_mat_table,epoch/accuracy,epoch/val_loss,_step,_wandb,_runtime,_timestamp,epoch/learning_rate,Chart_table
19,mobilenetv1_0.1_96_c3_o3_l1,1m1lxbx8,1m1lxbx8,NaN,NaN,NaN,NaN,NaN,NaN,mobilenetv1_0.1_96_c3_o3_l1,NaN,NaN,NaN,NaN,NaN,NaN,0,{'runtime': 16},9.475016,1.671308e+09,NaN,{'_latest_artifact_path': 'wandb-client-artifa...
14,sandy-forest-7,3oeqm2nt,3oeqm2nt,100.0,3.0,0.9,SGD,32.0,0.001,mobilenetv1_0.1_96_c3_o3_l1,0.020229,0.940476,60.0,"{'_type': 'table-file', 'ncols': 3, 'nrows': 9...",0.996042,0.221269,61,{'runtime': 259},260.234766,1.671308e+09,0.001,NaN
4,effortless-frost-17,1tkw3yhl,1tkw3yhl,100.0,3.0,0.9,SGD,32.0,0.001,mobilenetv1_0.1_96_c3_o3_l2,0.024092,0.972222,54.0,{'sha256': '216f09a4c259c1edb7bb9a5657ad753975...,0.992578,0.065462,55,{'runtime': 220},220.904655,1.671311e+09,0.001,NaN
9,mobilenetv1_0.1_96_c3_o3_l2,bc6hpasq,bc6hpasq,NaN,NaN,NaN,NaN,NaN,NaN,mobilenetv1_0.1_96_c3_o3_l2,NaN,NaN,NaN,NaN,NaN,NaN,0,{'runtime': 10},7.924718,1.671311e+09,NaN,{'_latest_artifact_path': 'wandb-client-artifa...
12,polished-salad-9,25l4qctj,25l4qctj,100.0,3.0,0.9,SGD,32.0,0.001,mobilenetv1_0.25_96_c3_o3_l1,0.018778,0.984127,82.0,{'sha256': 'dc096b7a36d7b936421eb1cf47c45512cb...,0.993073,0.058707,83,{'runtime': 333},333.955115,1.671309e+09,0.001,NaN


In [29]:
run.config.items()

dict_items([('id', '1m1lxbx8'), ('architecture', 'mobilenetv1_0.1_96_c3_o3_l1')])

In [30]:
run.config.get("id")

'1m1lxbx8'

In [32]:
run.config.get("architecture")

'mobilenetv1_0.1_96_c3_o3_l1'

In [46]:
path = run.summary.get("path")
path

In [52]:
run.history()(keys=[""])

,_step,Chart_table,_runtime,_timestamp
0,0,{'path': 'media/table/Chart_table_0_1fa1fcdfc7...,9.475016,1.671308e+09


In [19]:
run.summary._json_dict

{'_step': 0,
 '_wandb': {'runtime': 16},
 '_runtime': 9.475016117095947,
 '_timestamp': 1671307828.043302,
 'Chart_table': {'_latest_artifact_path': 'wandb-client-artifact://4cl5zcp4khzaymlrihxlie2imjzdqvj25w7luf1imma73rwgdu8vvx8pq3u6crll156b0wva92x061nrxom5rkzafsaeappsglk7tbb5feq84ups6uthss2f2i85afn2:latest/Chart_table.table.json',
  'path': 'media/table/Chart_table_0_1fa1fcdfc7cf2b004b0a.table.json',
  'size': 50,
  '_type': 'table-file',
  'ncols': 1,
  'nrows': 1,
  'sha256': '1fa1fcdfc7cf2b004b0af91e3403cecfd1c6c375e54c7e1c5efd416b20a294b4',
  'artifact_path': 'wandb-client-artifact://4cl5zcp4khzaymlrihxlie2imjzdqvj25w7luf1imma73rwgdu8vvx8pq3u6crll156b0wva92x061nrxom5rkzafsaeappsglk7tbb5feq84ups6uthss2f2i85afn2:latest/Chart_table.table.json'}}

In [16]:
runs_df

,summary,config,name
0,"{'epoch/loss': 0.011499449610710144, 'epoch/va...","{'id': '3ac53h9j', 'epochs': 100, 'classes': 3...",smooth-dew-21
1,"{'epoch/loss': 0.012104048393666744, 'epoch/ac...","{'id': '2w85dvzm', 'epochs': 100, 'classes': 3...",dandy-universe-20
2,"{'epoch/val_accuracy': 0.976190447807312, '_ru...","{'id': '2wgi6x5i', 'epochs': 100, 'classes': 3...",deft-fog-19
3,"{'epoch/epoch': 99, 'epoch/val_loss': 0.078373...","{'id': '1jqfxmbf', 'epochs': 100, 'classes': 3...",wobbly-brook-18
4,"{'_step': 55, '_wandb': {'runtime': 220}, 'epo...","{'id': '1tkw3yhl', 'epochs': 100, 'classes': 3...",effortless-frost-17
5,"{'_step': 0, '_wandb': {'runtime': 10}, '_runt...","{'id': '2csc8z41', 'architecture': 'mobilenetv...",mobilenetv1_0.5_96_c3_o3_l2
6,"{'_step': 0, '_wandb': {'runtime': 10}, '_runt...","{'id': '2tm8ysb4', 'architecture': 'mobilenetv...",mobilenetv1_0.3_96_c3_o3_l2
7,"{'_step': 0, '_wandb': {'runtime': 10}, '_runt...","{'id': '2a4a0fww', 'architecture': 'mobilenetv...",mobilenetv1_0.25_96_c3_o3_l2
8,"{'_wandb': {'runtime': 10}, '_runtime': 7.6740...","{'id': '277skkd9', 'architecture': 'mobilenetv...",mobilenetv1_0.2_96_c3_o3_l2
9,"{'_step': 0, '_wandb': {'runtime': 10}, '_runt...","{'id': 'bc6hpasq', 'architecture': 'mobilenetv...",mobilenetv1_0.1_96_c3_o3_l2


In [12]:
df_96_c1 = df.loc[(df['channels'] == 1) & (df['img_res'] == 96)]
df_96_c1

,img_res,classes,channels,alpha,tflite_INT8_model_size_kb,model_name,MACs,FLOPs,total_params,trainable_params,non_trainable_params,model_size_kb,tflite_model_size_kb,params_kb,dropout_rate


In [13]:
df_005_c1 = df.loc[(df['channels'] == 1) & (df['alpha'] == 0.05)]
df_005_c1

,img_res,classes,channels,alpha,tflite_INT8_model_size_kb,model_name,MACs,FLOPs,total_params,trainable_params,non_trainable_params,model_size_kb,tflite_model_size_kb,params_kb,dropout_rate


In [14]:
fig = px.bar(df_005_c1, x="img_res", y= "tflite_INT8_model_size_kb", text_auto=".2s",
    title=f"TFLite Model Size in kB - INT8")
#fig.add_hline(y=256000, line_width=3, line_dash="dash", line_color="orange", annotation_text="MCU constraint", 
              #annotation_position="bottom right")
#fig.update_layout(showlegend=True)
fig.update_layout(hovermode="y")
fig.show()

In [15]:
fig = px.scatter(df, x= "channels", y= "tflite_INT8_model_size_kb")
fig.update_layout(hovermode="y")
# fig.update_layout(
#     hoverlabel=dict(
#         bgcolor="white",
#         font_size=16,
#         font_family="Rockwell"
#     )
#)
fig.show()